<a href="https://colab.research.google.com/github/Bennibraun/Microwave-Analysis/blob/master/Particle_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, DotProduct
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PowerTransformer
from scipy.stats import multivariate_normal as mvn
from scipy import integrate
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

In [19]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1uLp4WsOXkQS76m0I61UrTibxMt-qSEtC'
downloaded = drive.CreateFile({'id': file_id })
downloaded.GetContentFile('all_data_yeast_particles.xlsx')

# Join single particle-frequency measurements into pseudo-multi-frequency particles

In [20]:
freqs = ['3.96GHz','7.65GHz','5.55GHz','2.38GHz','0.8GHz']
types = ['5.5um','3um']

cols = ['Type']
for freq in freqs:
  cols += [freq+'_Re']
  cols += [freq+'_Im']

df = pd.DataFrame(columns=cols)

# Gather particle measurements from all sheets together
# Scrap time measurements for the pseudo-combination of frequencies
for freq in freqs:
  for type in range(len(types)):
    newdf = pd.read_excel('all_data_yeast_particles.xlsx',
                          header=None,
                          names=['Time','Epsilon_Re','Epsilon_Im'],
                          sheet_name=freq,
                          skiprows=2,
                          usecols=range(12+(type*3),15+(type*3)))
  
    newdf.dropna(subset=['Time','Epsilon_Re','Epsilon_Im'],inplace=True)

    newdf['Type'] = types[type]

    df['Type'] = pd.Series(newdf['Type'].values)
    df[freq+'_Re'] = pd.Series(newdf['Epsilon_Re'].values)
    df[freq+'_Im'] = pd.Series(newdf['Epsilon_Im'].values)

# # There is an unequal quantity of measurements for each frequency
# # Since the multi-frequency measurements are falsely associated anyway,
# # We can now randomly duplicate readings to fill in the gaps
# num_rows = len(df[cols[0]])
# for col in cols:
#   # Find how many rows in the col are non-empty
#   # So that random rows are selected only from the original set
#   # And not from rows that have previously been randomly filled
#   nonempty_rows = num_rows - len(df[col].index[df[col].isnull()])

#   while (df[col].isnull().values.any()):
#     # Get random row
#     rand_row = df[col].head(nonempty_rows).sample()
#     index = rand_row.index[0]

#     # Place this random value into the next empty cell in the column
#     next_nan = df[col].index[df[col].isnull()][0]
#     df.at[next_nan,col] = rand_row.values[0]

pd.set_option('display.max_rows',200)
df

5.5um
3um
5.5um
3um
5.5um
3um
5.5um
3um
5.5um
3um


,Type,3.96GHz_Re,3.96GHz_Im,7.65GHz_Re,7.65GHz_Im,5.55GHz_Re,5.55GHz_Im,2.38GHz_Re,2.38GHz_Im,0.8GHz_Re,0.8GHz_Im
0,3um,0.198,0.0,0.146,0.069,0.150,0.0,0.119,0.0,0.17,0.0
1,3um,0.112,0.0,0.163,0.084,0.145,0.0,0.139,0.0,0.20,0.0
2,3um,0.112,0.0,0.125,0.077,0.133,0.0,0.110,0.0,0.16,0.0
3,3um,0.224,0.0,0.125,0.117,0.155,0.0,0.126,0.0,0.16,0.0
4,3um,0.068,0.0,0.133,0.105,0.152,0.0,0.117,0.0,0.16,0.0
5,3um,0.123,0.0,0.214,0.092,0.159,0.0,0.149,0.0,0.16,0.0
6,3um,0.108,0.0,0.156,0.091,0.150,0.0,0.125,0.0,0.15,0.0
7,3um,0.117,0.0,0.116,0.100,0.197,0.0,0.102,0.0,0.20,0.0
8,3um,0.117,0.0,0.159,0.067,0.127,0.0,0.120,0.0,0.17,0.0
9,3um,0.119,0.0,0.197,0.068,0.160,0.0,0.120,0.0,0.15,0.0


# Fit classifiers

In [21]:
# Potentially do QDA, but unsure

# clfr = QDA(store_covariance=True)